In [5]:
%pip install -r requirements.txt

  Using cached python_docx-1.1.2-py3-none-any.whl.metadata (2.0 kB)
  Using cached sentence_transformers-3.0.1-py3-none-any.whl.metadata (10 kB)
  Using cached elasticsearch-8.14.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached huggingface_hub-0.24.5-py3-none-any.whl.metadata (13 kB)
  Using cached docx-0.2.4-py3-none-any.whl
  Using cached elastic_transport-8.17.1-py3-none-any.whl.metadata (3.8 kB)
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   -- ------------------------------------- 0.8/11.5 MB 6.7 MB/s eta 0:00:02
   -------- ------------------------------- 2.4/11.5 MB 7.9 MB/s eta 0:00:02
   ------------ --------------------------- 3.7/11.5 MB 7.0 MB/s eta 0:00:02
   ------------------ --------------------- 5.2/11.5 MB 6.9 MB/s eta 0:00:01
   ----------------------- ---------------- 6.8/11.5 MB 7.0 MB/s eta 0:00:01
   ----------------------------- ---------- 8.4/11.5 MB 7.0 MB/s eta 0:00:01
   ---------------------------------- ----- 10.0/11.5 MB 7.1 M

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
vllm 0.7.3 requires transformers>=4.48.2, but you have transformers 4.46.3 which is incompatible.


In [18]:
from docx import Document
import pandas as pd
import glob
import os
import tqdm

In [46]:
def get_data(files):
    sections = []
    subsections = []
    questions = []
    answers = []
    for file in files:
        section = os.path.basename(file)[:-5].replace('_', ' ')  # name of the section
        print(f'Section {section}')
        doc = Document(file)
        current_answer = ''
        current_subsection = None
        
        for par in tqdm.tqdm(doc.paragraphs):
            if par.text.strip() == '':
                continue
            if par.style.name == 'Heading 1':  # subsection started
                current_subsection = par.text
            elif par.style.name == 'Heading 2':  #question started
                if current_answer:
                    answers.append(current_answer)
                questions.append(par.text)
                subsections.append(current_subsection)
                sections.append(section)
                current_answer = ''
            else:
                current_answer += par.text + '\n'
        answers.append(current_answer)
    return pd.DataFrame({'section': sections, 'subsection': subsections, 'question': questions, 'answer': answers})

In [47]:
files = ['docs/Classical_models.docx', 'docs/Classical_NLP.docx', 
         'docs/Data.docx', 'docs/Deep_Learning.docx', 'docs/LLM.docx', 
         'docs/Metrics.docx', 'docs/Probability_and_Statistics.docx']

In [48]:
ds = get_data(files)
ds

Section Classical models


100%|██████████████████████████████████████████████████████████████████████████████| 978/978 [00:00<00:00, 1611.65it/s]


Section Classical NLP


100%|██████████████████████████████████████████████████████████████████████████████| 403/403 [00:00<00:00, 1236.55it/s]


Section Data


...

Section Metrics


100%|██████████████████████████████████████████████████████████████████████████████| 498/498 [00:00<00:00, 1545.33it/s]


Section Probability and Statistics


100%|██████████████████████████████████████████████████████████████████████████████| 567/567 [00:00<00:00, 1430.24it/s]


,section,subsection,question,answer
0,Classical models,Linear Regression,What Is a Linear Regression Model? List Its Dr...,A linear regression model is a model in which ...
1,Classical models,Linear Regression,What are various assumptions used in linear re...,Linear regression is done under the following ...
2,Classical models,Linear Regression,What methods for solving linear regression do ...,"To solve linear regression, you need to find t..."
3,Classical models,Linear Regression,Maximum Likelihood Estimation for Linear Regre...,Linear regression can be written as a CPD in t...
4,Classical models,Linear Regression,Ordinary least squares,The ordinary least squares (OLS) method can be...
...,...,...,...,...
520,Probability and Statistics,Miscellaneous,Simpson’s paradox,Simpson’s paradox refers to the situations in ...
521,Probability and Statistics,Miscellaneous,What Is a Statistical Interaction?,A statistical interaction is when two or more ...
522,Probability and Statistics,Miscellaneous,How do you identify if a coin is biased?,We collect data by flipping the coin 200 times...
523,Probability and Statistics,Miscellaneous,What does Design of Experiments mean?,"Design of experiments also known as DOE, it is..."


In [49]:
ds['question'][2]

'What methods for solving linear regression do you know?'

In [50]:
ds['answer'][1]

'Linear regression is done under the following assumptions:\nThe sample data used for modeling represents the entire population.\nThere exists a linear relationship between the X-axis variable and the mean of the Y variable.\nThe residual variance is the same for any X values. This is called homoscedasticity. Residual Variance\xa0(also called\xa0unexplained variance\xa0or\xa0error variance) is the variance of any\xa0error (residual).\nThe errors or residuals of the data are normally distributed and independent from each other.\xa0\nThere is minimal multicollinearity between explanatory variables\xa0\nExtreme violations of the above assumptions lead to redundant results. Smaller violations of these result in greater variance or bias of the estimates.\n'

In [51]:
ds_cleaned = ds.replace({'\u200d': '', '\xa0': ' ', '\u200b': ''}, regex=True)  # get rid of Unicode codes

In [52]:
ds_cleaned['question'][2]

'What methods for solving linear regression do you know?'

In [53]:
ds_cleaned['answer'][1]

'Linear regression is done under the following assumptions:\nThe sample data used for modeling represents the entire population.\nThere exists a linear relationship between the X-axis variable and the mean of the Y variable.\nThe residual variance is the same for any X values. This is called homoscedasticity. Residual Variance (also called unexplained variance or error variance) is the variance of any error (residual).\nThe errors or residuals of the data are normally distributed and independent from each other. \nThere is minimal multicollinearity between explanatory variables \nExtreme violations of the above assumptions lead to redundant results. Smaller violations of these result in greater variance or bias of the estimates.\n'

In [54]:
ds_cleaned.to_csv('data/db.csv', index=False)

In [ ]:
from datasets import Dataset

comments_dataset = Dataset.from_pandas(ds)
comments_dataset